## Image classification with CNN Quickstart

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# This file contains all the main external libs we'll use
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
PATH = "data/dogscats/"
sz=224
arch=vgg16
bs=64

In [4]:
# Uncomment the below if you need to reset your precomputed activations
# !rm -rf {PATH}tmp

In [5]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz))

In [6]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /home/paperspace/.torch/models/vgg16_bn-6c64b313.pth
100%|██████████| 553507836/553507836 [00:13<00:00, 40366039.98it/s]


100%|██████████| 32/32 [00:25<00:00,  1.24it/s]


In [7]:
learn.fit(0.01, 3, cycle_len=1)

epoch      trn_loss   val_loss   accuracy                     
    0      0.069592   0.026705   0.992     
    1      0.060043   0.027876   0.9905                       
    2      0.063669   0.030901   0.9895                       



[array([0.0309]), 0.9895]

In [8]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)

In [9]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs, num_workers=4)
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [10]:
learn.fit(1e-2, 2)

epoch      trn_loss   val_loss   accuracy                     
    0      0.06825    0.033463   0.9905    
    1      0.045674   0.040923   0.989                        



[array([0.04092]), 0.989]

In [11]:
learn.precompute=False

In [12]:
learn.fit(1e-2, 1, cycle_len=1)

epoch      trn_loss   val_loss   accuracy                     
    0      0.045885   0.025537   0.991     



[array([0.02554]), 0.991]

In [13]:
learn.unfreeze()

In [14]:
lr=np.array([1e-4,1e-3,1e-2])

In [15]:
learn.fit(lr, 1, cycle_len=1)

  0%|          | 0/360 [00:00<?, ?it/s]


RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1518244421288/work/torch/lib/THC/generic/THCStorage.cu:58

In [ ]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [ ]:
learn.fit(lr, 3, cycle_len=3)

In [ ]:
log_preds,y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)accuracy_np(probs,y)